# Computing Invariant Ideals

As mentioned in the README.md Polar can compute invariants among moments of program variables.
Given some moments Polar produces a list of invariants among them. Moreover, these invariants form a basis for the ideal of all polynomial invariants among the given moments.

# Command Line Interface

To produce the invariants, we pass the moments we want to consider in the `goals` parameter and additionally use the `invariants` flag.

In [1]:
!python ../polar.py loops/loop.prob --goals "E(x)" "E(y)" "E(x**2)" --invariants


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

E(x) = 1; n**3/256 + 133*n**2/256 + 205*n/128 + 1
Solution is exact

E(y) = 0; -n/8
Solution is exact

E(x**2) = 1; n**6/65536 + 685*n**5/65536 + 143911*n**4/196608 + 641881*n**3/196608 + 645391*n**2/98304 + 210187*n/24576 + 1
Solution is exact


-------------------
-   Invariants    -
-------------------

Following is a gröbner basis for the invariant ideal:

E(x) + 2*E(y)**3 - 133*E(y)**2/4 + 205*E(y)/16 - 1 = 0

-E(x**2)/4 + E(y)**6 - 685*E(y)**5/8 + 143911*E(y)*

Polar now outputs the closed-form formulas of the moments followed by a basis for the ideal of *all* polynomial invariants.
If we want the invariants for a non-probabilistic loop, we can omit the goals parameter and get a basis for *all* polynomial invariants.

In [2]:
!python ../polar.py loops/fibonacci2.prob --invariants


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

z = 1; (-1)**n
Solution is exact

a = 0; 0; 1; sqrt(5)*(1/2 - sqrt(5)/2)**n/10 + (1/2 - sqrt(5)/2)**n/2 - sqrt(5)*(1/2 + sqrt(5)/2)**n/10 + (1/2 + sqrt(5)/2)**n/2
Solution is exact

b = 0; -sqrt(5)*(1/2 - sqrt(5)/2)**n/5 + sqrt(5)*(1/2 + sqrt(5)/2)**n/5
Solution is exact

c = 1; 1; 2; -sqrt(5)*(1/2 - sqrt(5)/2)**n/10 + (1/2 - sqrt(5)/2)**n/2 + sqrt(5)*(1/2 + sqrt(5)/2)**n/10 + (1/2 + sqrt(5)/2)**n/2
Solution is exact


-------------------
-   Invariants    -
-------

# In Python

We can also compute invariants using Python by importing the functionality in Polar. We first have to compute the closed-forms and then construct the invariant ideal:

In [3]:
from inputparser import Parser
from program import normalize_program
from recurrences import RecBuilder
from recurrences.solver import RecurrenceSolver
from invariants import InvariantIdeal

program = Parser().parse_file("loops/loop.prob")
# Construct normal form so that Polar can analyze it
program = normalize_program(program)

# Construct and solve recurrences
rec_builder = RecBuilder(program)
monomials = ["x", "y", "x**2"]
closed_forms = {}
for monomial in monomials:
    # Construct the recurrences describing E(monomial) -> expected value of monomial
    recurrences = rec_builder.get_recurrences(monomial)
    # solve and save the closed-forms (use E(monomial) as the id because the loop is probabilistic)
    closed_forms[f"E({monomial})"] = RecurrenceSolver(recurrences).get(monomial)

# Construct the invariant ideal
invariant_ideal = InvariantIdeal(closed_forms)
basis = invariant_ideal.compute_basis()
print(basis)

{-E(x**2)/4 + E(y)**6 - 685*E(y)**5/8 + 143911*E(y)**4/192 - 641881*E(y)**3/1536 + 645391*E(y)**2/6144 - 210187*E(y)/12288 + 1/4, E(x) + 2*E(y)**3 - 133*E(y)**2/4 + 205*E(y)/16 - 1}


The basis contains two elements that capture all invariants among the expected values of `x`, `y` and `x**2`. We can perform the same computation for our non-probabilistic loop to get a basis for the invariant ideal.

In [4]:
from inputparser import Parser
from program import normalize_program
from recurrences import RecBuilder
from recurrences.solver import RecurrenceSolver
from invariants import InvariantIdeal

program = Parser().parse_file("loops/fibonacci2.prob")
# Construct normal form so that Polar can analyze it
program = normalize_program(program)

# Construct and solve recurrences
rec_builder = RecBuilder(program)
closed_forms = {}
# We only need to consider variables because the loop is non-probabilistic
for variable in program.original_variables:
    # Construct the recurrences describing the values of "variable"
    recurrences = rec_builder.get_recurrences(variable)
    # solve and save the closed-forms
    closed_forms[str(variable)] = RecurrenceSolver(recurrences).get(variable)

# Construct the invariant ideal
invariant_ideal = InvariantIdeal(closed_forms)
basis = invariant_ideal.compute_basis()
print(basis)

{b**2 + b*c - c**2 + z, b**4 + 2*b**3*c - b**2*c**2 - 2*b*c**3 + c**4 - 1, a + b - c}
